In [ ]:
USER_FLAG = "--user"

In [ ]:
! pip install {USER_FLAG} --upgrade google-cloud-aiplatform

In [ ]:
! pip install {USER_FLAG} --upgrade google-cloud-storage

In [1]:
import pandas as pd
df=pd.read_csv('gs://laah-allegis/hackathon_data.csv')

In [ ]:
df.head()

In [2]:
columns=list(df.columns.values)

In [3]:
new_columns = [s.replace(" ", "_") for s in columns]

In [ ]:
new_columns

In [4]:
df2=pd.read_csv('gs://laah-allegis/hackathon_data.csv', names=new_columns, skiprows=2)

In [6]:

#Use pd.to_datetime():

df2['WEEK_ENDING_DATE'] = pd.to_datetime(df2['WEEK_ENDING_DATE'])

In [5]:
df2=df2.drop(['DMONAME'], axis=1)

In [7]:
df2.head()

,DIVISION_NAME,WEEK_ENDING_DATE,PROD_STAFF_CONTRACTOR_HC_AMT,RECRUITER_TOTAL_CONTACTS_AMT,RETAIL_CONTRACTOR_HC_AMT,RECRUIT_CONTRACTOR_HC_AMT,RECRUITER_ACCOUNT_INTERVIEWS_AMT,PRODUCER_HC,RECRUITER_EMAI_LCONTACTS_AMT,RECRUITER_SERVICE_TOUCHPOINTS,...,OFFER_ACCEPT_AMT,CLIENT_INTERVIEW_AMT,STAFF_SPREAD_AMT,RECRUITER_G2S_MODIFIED_AMT,NON_RECRUIT_CONTRACTOR_HC_AMT,DPT_CONTRACTOR_HC_AMT,SALES_COMPLETED_ACTIVITIES_AMT,RECRUITER_TOTAL_SUBMITTALS_AMT,CORP_ACCTS_AMT,RETAIL_FINISHES_AMT
0,3,2017-01-07,185,497,163,27,4,19,2,2,...,12,12,63568.14,209,158,16,44,11,52,3
1,9,2017-01-07,57,0,53,0,0,0,0,0,...,0,0,7748.79,0,57,1,0,0,22,3
2,4,2017-01-07,33,224,33,0,2,8,2,2,...,2,1,6028.41,124,33,0,20,14,12,4
3,7,2017-01-07,299,569,267,0,7,16,10,22,...,15,14,58125.32,190,299,8,20,32,77,22
4,0,2017-01-07,30,255,30,0,1,14,2,1,...,1,2,14094.62,113,30,15,157,6,25,2


In [ ]:
df2.describe()

In [ ]:
df2['WEEK_ENDING_DATE']

In [ ]:
df2.dtypes

In [8]:
df2.to_csv('gs://laah-allegis/new_allegis.csv', index=False)

In [9]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  laah-vertex-pipeline


In [10]:
BUCKET_NAME = "gs://laah-allegis"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

In [11]:
import os

from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

In [12]:

ds = dataset = aiplatform.TabularDataset.create(
    display_name="allegis2-tabular-dataset",
    gcs_source='gs://laah-allegis/new_allegis.csv',
)

ds.resource_name

INFO:google.cloud.aiplatform.datasets.dataset:Creating TabularDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create TabularDataset backing LRO: projects/1052547690175/locations/us-central1/datasets/3375228018388631552/operations/8146128886795599872
INFO:google.cloud.aiplatform.datasets.dataset:TabularDataset created. Resource name: projects/1052547690175/locations/us-central1/datasets/3375228018388631552
INFO:google.cloud.aiplatform.datasets.dataset:To use this TabularDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TabularDataset('projects/1052547690175/locations/us-central1/datasets/3375228018388631552')


'projects/1052547690175/locations/us-central1/datasets/3375228018388631552'

In [13]:
job = aiplatform.AutoMLTabularTrainingJob(
    display_name="train-allegis-automl-2",
    optimization_prediction_type="regression",
)

# This will take around an hour to run
model = job.run(
    dataset=ds,
    target_column="STAFF_SPREAD_AMT",
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    model_display_name="allegis-prediction-model",
    disable_early_stopping=False,
    sync=False
)

INFO:google.cloud.aiplatform.training_jobs:No column transformations provided, so now retrieving columns from dataset in order to set default column transformations.
INFO:google.cloud.aiplatform.training_jobs:The column transformation of type 'auto' was set for the following columns: ['FISCAL_MONTH', 'PRODUCER_HC', 'SALES_MEETINGS_AMT', 'CORP_ACCTS_AMT', 'PROD_STAFF_CONTRACTOR_HC_AMT', 'SERVICES_CONTRACTOR_HC_AMT', 'CLIENT_INTERVIEW_AMT', 'NON_RECRUIT_CONTRACTOR_HC_AMT', 'RECRUITER_TOTAL_SUBMITTALS_AMT', 'RECRUITER_G2S_MODIFIED_AMT', 'DPT_CONTRACTOR_HC_AMT', 'CONTRACTOR_HC_AMT', 'RETAIL_FINISHES_AMT', 'DIVISION_NAME', 'STAFF_CONTRACTOR_HC_AMT', 'RETAIL_CONTRACTOR_HC_AMT', 'RECRUITER_ACCOUNT_INTERVIEWS_AMT', 'RECRUITER_SERVICE_TOUCHPOINTS', 'RECRUITER_TOTAL_CONTACTS_AMT', 'SSO_CONTRACTOR_HC_AMT', 'RECRUIT_CONTRACTOR_HC_AMT', 'RECRUITER_OFFERS_ACCEPTED_AMT', 'LARGE_CORP_ACCTS_AMT', 'RECRUITER_EMAI_LCONTACTS_AMT', 'SALES_COMPLETED_ACTIVITIES_AMT', 'WEEK_ENDING_DATE', 'OFFER_ACCEPT_AMT'].
